## Install the following libraries

1. **PyTube:** To faciliate YouTube video downloads and manipulation.

```jupyter
! pip install pytube
```
<br>

2. **PyDub:** To import and manipulate audio files.

```jupyter
! pip install pydub
```
<br>

**Dcoumentation:**
- PyTube Documentation --> [click here](https://pytube.io/en/latest/index.html)
- PyDub Documentation --> [click here](https://github.com/jiaaro/pydub)

## Download the executable files for FFmpeg
*The purpose of FFmpeg is to "record, convert and stream audio and video"*

1. Follow this link [here](https://ffmpeg.org/download.html) to download the packages and executables.
<br>

2. Download the appropriate zipped folder for your operating system.
<br>

3. Unzip/Extract the files.
<br>

4. It's important to note that for some reason not discussed in the pydub documentation, the following executable files should be placed in the script/notebook directory:
    - ffmpeg.exe
    - ffprobe.exe
    - ffplay.exe

In [2]:
import os

from pytube import YouTube

#class from the pydub package is imported to manipulate audio files, such as slicing and exporting.
from pydub import AudioSegment

# after much debugging, this seems important:
AudioSegment.converter = r'C:\Users\Administrator\Desktop\repos\auto_caption\ffmpeg.exe'                        
# AudioSegment.ffprobe   = r'C:\Users\Administrator\Desktop\repos\auto_caption\ffprobe.exe'

In [3]:
# URL of a trial-video
URL = 'https://www.youtube.com/watch?v=xR7qcezLcXI'

# create a YouTube object from pytube library
yt = YouTube(URL)

#The audio stream of the YouTube video is filtered and the first available audio stream is selected.
# this could all be condenced into one line of code:
# first_audio_stream = youtube.streams.filter(only_audio=True).first()
video_stream_query = yt.streams
audio = video_stream_query.filter(only_audio=True)
first_audio_stream = audio.first()  # returns Stream object (or None)

trial_audio_filepath = first_audio_stream.download(filename='trial_audio.mp4')

In [4]:
# Load audio from mp4 file...  
audio = AudioSegment.from_file(trial_audio_filepath, format="mp4")

# slice audio
one_minute = 1 * 60 * 1_000  # number of min desired for this trial (in ms)
first_minute = audio[:one_minute]

first_minute_filename = 'first_minute_audio.wav'

# Export the audio
first_minute.export(first_minute_filename, format="wav")

<_io.BufferedRandom name='first_minute_audio.wav'>

In [5]:
import subprocess

input_filename = first_minute_filename
output_filename = "eng_10_16k.wav"

#The command list is created, which represents the command to be executed using FFmpeg. 
# It specifies the input file, the desired audio sample rate of 16000 Hz, and the output file.

command = ['ffmpeg', '-y', '-i', input_filename, '-ar', '16000', output_filename]

#The subprocess.run() function is called to execute the FFmpeg command. 
#This command will resample the input audio file to a sample rate of 16000 Hz and save it as the output file.

subprocess.run(command)

CompletedProcess(args=['ffmpeg', '-y', '-i', 'first_minute_audio.wav', '-ar', '16000', 'eng_10_16k.wav'], returncode=0)

In [6]:
!mkdir "temp_dir"

#git clone command. 
#It fetches the source code and creates a local copy of the repository on your machine.

!git clone https://github.com/pytorch/fairseq

# Change current working directory
#The !pwd command prints the current working directory, displaying the current path. 
!pwd

#The %cd command changes the current working directory to "/content/fairseq".

%cd "/content/fairseq"

#The --editable flag indicates that the package should be installed in editable mode, 
#allowing modifications to the package source code without needing to reinstall it.
#./ refers to the current directory
!pip install --editable ./ 

#tensorboardX is a library that provides a wrapper around TensorFlow's 
#TensorBoard, enabling visualization and logging of training progress and results.

!pip install tensorboardX


Cloning into 'fairseq'...


[WinError 2] The system cannot find the file specified: '/content/fairseq'
C:\Users\Administrator\Desktop\repos\auto_caption


'pwd' is not recognized as an internal or external command,
operable program or batch file.


Obtaining file:///C:/Users/Administrator/Desktop/repos/auto_caption


ERROR: file:///C:/Users/Administrator/Desktop/repos/auto_caption does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


   ---------------------------------------- 0.0/101.7 kB ? eta -:--:--
   --------------- ----------------------- 41.0/101.7 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 101.7/101.7 kB 1.2 MB/s eta 0:00:00


In [ ]:
# use colab free
# MMS-1B:FL102 model - 102 Languages - FLEURS Dataset
#!wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_fl102.pt'


In [8]:
# use high RAM, such as colab pro
# # MMS-1B-all - 1162 Languages - MMS-lab + FLEURS + CV + VP + MLS
! wget -P ./models_new 'https://dl.fbaipublicfiles.com/mms/asr/mms1b_all.pt'

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import subprocess
import os


# Set the value of the environment variable "TMPDIR"
os.environ["TMPDIR"] = '/content/temp_dir'

# Set the value of the environment variable "PYTHONPATH"
os.environ["PYTHONPATH"] = "."

# Set the value of the environment variable "PREFIX"
os.environ["PREFIX"] = "INFER"

# Set the value of the environment variable "HYDRA_FULL_ERROR"
os.environ["HYDRA_FULL_ERROR"] = "1"

# Set the value of the environment variable "USER"
os.environ["USER"] = "micro"

filename = "/content/chunk2/eng_10_16k.wav"

# Open a file named "output.txt" in write mode
with open('/content/chunk2/output.txt', 'w') as f:

    # Command to run MMS inference
    command = ['python', 'examples/mms/asr/infer/mms_infer.py', '--model', '/content/fairseq/models_new/mms1b_all.pt', '--lang', 'eng', '--audio', filename]

    # Run the command and redirect the standard output to the file
    subprocess.run(command, stdout=f)




In [ ]:
import os
from pytube import YouTube
from pydub import AudioSegment
import subprocess

# URL of the video
url = 'https://www.youtube.com/watch?v=T1WAHb47ChI&t=60s'

youtube = YouTube(url)
video = youtube.streams.filter(only_audio=True).first()
out_file = video.download(output_path="/content/")

# Load the audio
audio = AudioSegment.from_file(out_file, format="mp4")

# Create the directory if it doesn't exist
os.makedirs('/content/chunk4', exist_ok=True)

# Determine the end time for slicing in milliseconds
end_time = 3*60*1000  # 3 minutes

# Initialize start time and chunk size in milliseconds
start_time = 0
chunk_size = 60*1000  # 60 seconds

# Slice and export each 60-second chunk of audio
i = 0
while start_time < end_time:
    # Calculate the end time for the current slice
    slice_end_time = min(start_time + chunk_size, end_time)
    
    # Slice the audio
    sliced_audio = audio[start_time:slice_end_time]
    
    # Export the sliced audio
    original_filename = f"/content/chunk4/audio_{i}.wav"
    sliced_audio.export(original_filename, format="wav")
    
    # Apply the ffmpeg process
    output_filename = f"/content/chunk4/urdu_{i}_16k.wav"  # Updated filename
    # convert to 16000 hz sample rate
    command = ['ffmpeg', '-y', '-i', original_filename, '-ar', '16000', output_filename]
    subprocess.run(command)
    
    # Move to the next slice
    start_time += chunk_size
    i += 1


In [ ]:
import os
import subprocess

# Set environment variables
os.environ["TMPDIR"] = '/content/temp_dir'
os.environ["PYTHONPATH"] = "."
os.environ["PREFIX"] = "INFER"
os.environ["HYDRA_FULL_ERROR"] = "1"
os.environ["USER"] = "micro"

# Define your directory
directory = '/content/chunk4/'

# Define the range of file indices
file_indices = range(3)  # This will give you the numbers 0 to 2

# Define a single output file to store all transcriptions
combined_output_filename = os.path.join(directory, 'combined_transcription.txt')

# Check if combined transcription file already exists and if so, remove it
if os.path.exists(combined_output_filename):
    os.remove(combined_output_filename)

# Loop through the file indices
for i in file_indices:
    # Construct the input file name
    input_filename = os.path.join(directory, f'eng_{i}_16k.wav')

    # Check if file exists
    if not os.path.exists(input_filename):
        print(f"File {input_filename} does not exist, skipping.")
        continue

    # Define the output filename for the transcription
    output_filename = os.path.join(directory, f'transcription_{i}.txt')

    # Open the transcription file in write mode
    with open(output_filename, 'w') as f:
        # Run the model inference on the audio file and save the output to the file
        subprocess.run(['python', 'examples/mms/asr/infer/mms_infer.py', '--model', '/content/fairseq/models_new/mms1b_all.pt', '--lang', 'eng', '--audio', input_filename], stdout=f)

    # Now, append the individual transcription to the combined file
    with open(output_filename, 'r') as f_in, open(combined_output_filename, 'a') as f_out:
        # Read the individual transcription and write it to the combined file
        f_out.write(f_in.read())
        # Optionally, write a newline character to separate the transcriptions
        f_out.write('\n')
